In [2]:
import fitz  # PyMuPDF
import re
import os
import csv
from datetime import datetime

def extraer_campos_verticales(pdf_path):
    # el re lo uso para limpiar el texto, creo esta funcion con el fin de extraer de manera vertical los campos
    # y no horizontal como lo hace la otra
    doc = fitz.open(pdf_path)
    texto_completo = ""
    
    for pagina in doc:
        texto_completo += pagina.get_text("text") + "\n"
    
    doc.close()
    
    resultados = {}
    
    def extraer_valor_debajo(campo, texto, lineas_abajo=1):
        lineas = texto.split('\n')
        for i, linea in enumerate(lineas):
            if campo in linea:
                if i + lineas_abajo < len(lineas):
                    valor = lineas[i + lineas_abajo].strip()
                    if any(x in campo for x in ['Total', 'SALDO', 'IVA', 'Ret.', 'Percep.']):
                        valor = re.sub(r'[^\d.,]', '', valor).replace('.', '').replace(',', '.')
                        try:
                            return float(valor) if '.' in valor else int(valor)
                        except:
                            return valor
                    return valor
        return "No encontrado"
    # estos son los saltos de linea que doy en cada uno de manera vertical
    # para que no se confunda con los horizontales
    campos_verticales = {
        "FECHA DE EMISION:": 22,
        "PAGADOR:": 22,
        "Nº DE CUIT:": 22,
        "TOTAL PRESENTADO $": 29,
        "TOTAL DESCUENTO": 29,
        "SALDO $": 29
    }
    
    for campo, lineas_abajo in campos_verticales.items():
        resultados[campo] = extraer_valor_debajo(campo, texto_completo, lineas_abajo)
    
    resultados["Razón Social"] = extraer_valor_debajo("Razón Social", texto_completo, 1)
    resultados["Establecimiento"] = extraer_valor_debajo("Establecimiento", texto_completo, 1)
    
    # uso el replace de las comas para que no me tome el punto como separador de miles y la coma como separador decimal
    def extraer_valor_monetario(patron, texto):
        match = re.search(patron, texto)
        if match:
            valor = match.group(1).replace('.', '').replace(',', '.')
            try:
                return float(valor)
            except:
                return match.group(1).strip()
        return "No encontrado"
    
    resultados["IVA 21.00%"] = extraer_valor_monetario(r'IVA\s*21,?00\s*%?\s*\$\s*([\d.,]+)', texto_completo)
    resultados["Ret.IB CAP. FED."] = extraer_valor_monetario(r'Ret.IB CAP.FED.\s*[\d.,]+\s*%?\s*\$\s*([\d.,]+)', texto_completo)
    resultados["Percep. AFIP"] = extraer_valor_monetario(r'Percep./Retenc.AFIP\s*[-]?\s*DGI\s*\$\s*([\d.,]+)', texto_completo)
    
    mapeo_nombres = {
        "FECHA DE EMISION:": "FECHA DE EMISION",
        "PAGADOR:": "PAGADOR",
        "Nº DE CUIT:": "Nº DE CUIT",
        "TOTAL DESCUENTO": "Total Descuento",
        "SALDO $": "Saldo",
        "TOTAL PRESENTADO $": "Total Presentado",
        "IVA 21.00%": "IVA",
        "Ret.IB CAP. FED.": "Retención IB",
        "Percep. AFIP": "Percepción AFIP"
    }
    
    return {mapeo_nombres.get(k, k): v for k, v in resultados.items()}


carpeta_pdfs = "C:/Users/gesti/Downloads/Tarjetas/"
archivo_csv = "resultados_extracion.csv"


campos_csv = [
    "FECHA DE EMISION",
    "PAGADOR",
    "Nº DE CUIT",
    "Razón Social",
    "Establecimiento",
    "Total Presentado",
    "Total Descuento",
    "Saldo",
    "IVA",
    "Retención IB",
    "Percepción AFIP",
    "Archivo PDF"  
]


with open(archivo_csv, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=campos_csv)
    writer.writeheader()
    
    for archivo in os.listdir(carpeta_pdfs):
        if archivo.lower().endswith('.pdf'):
            try:
                pdf_path = os.path.join(carpeta_pdfs, archivo)
                print(f"Procesando: {archivo}")
                
                datos = extraer_campos_verticales(pdf_path)
                
                
                datos["Archivo PDF"] = archivo
                
               
                writer.writerow(datos)
                
            except Exception as e:
                print(f"Error al procesar {archivo}: {str(e)}")
                
                fila_error = {campo: f"ERROR: {str(e)}" for campo in campos_csv}
                fila_error["Archivo PDF"] = archivo
                writer.writerow(fila_error)

print(f"\nProceso completado. Resultados guardados en: {archivo_csv}")

PermissionError: [Errno 13] Permission denied: 'resultados_extracion.csv'

In [11]:
pip uninstall fitz


Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pymupdf


Note: you may need to restart the kernel to use updated packages.
